In [35]:
import os, warnings, sys, yaml
from stream2segment.process.db import get_session
from stream2segment.io.db.models import Station, Segment, concat, Channel
from stream2segment.io.utils import loads_inv
# # warnings.simplefilter('always')
# _fle = os.path.abspath(os.path.join('.', 'jnconfig.yaml'))
# assert os.path.isfile(_fle)
# with open(_fle, "r") as _:
#     jnconfig = yaml.safe_load(_)

# # dbpath_old = jnconfig['dbpath_old']
# dbpath = jnconfig['dbpath_new']

import sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from sod.core.dataset import open_dataset

In [36]:
import pandas as pd
_dfr = pd.read_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/globalset.hdf')
_dfr_loc_d1 = pd.read_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/oneminutewindows.hdf.loccode.hdf')
_dfr_loc_d2 = pd.read_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/magnitudeenergy.hdf.loccode.hdf')
_dfr_chile = pd.read_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/cx_chile.hdf')

_dfr_d1 = pd.read_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/oneminutewindows_sn_only.hdf')
_dfr_d2 = pd.read_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/magnitudeenergy.hdf')


# dataframe_old = dataframe.copy()
# dataframe = open_dataset('globalset', normalize=False)

In [37]:
dfr = _dfr.copy()
dfr_loc_d1 = _dfr_loc_d1.copy()
dfr_loc_d2 = _dfr_loc_d2.copy()
dfr_chile = _dfr_chile.copy()
dfr_d1 = _dfr_d1.copy()
dfr_d2 = _dfr_d2.copy()

In [38]:
# some assertions:
ID = 'Segment.db.id'  #'globalset.id'
LOC = 'location_code'

assert LOC not in dfr.columns
assert LOC in dfr_loc_d1.columns
assert LOC in dfr_loc_d2.columns
assert LOC in dfr_chile.columns

assert ID in dfr.columns
assert ID in dfr_loc_d1.columns
assert ID in dfr_loc_d2.columns
assert ID in dfr_chile.columns

print('\nglobalset columns:')
print(dfr.columns)
print('\nloccha oneminutewindows columns')
print(dfr_loc_d1.columns)
print('\nloccha magnitudeenergy columns')
print(dfr_loc_d2.columns)
print('\nchile columns')
print(dfr_chile.columns)

col1 = (_ for _ in dfr.columns if _ not in ('subclass', 'window_type', 'dataset_id'))
col2 = (_ for _ in dfr_chile.columns if _ != LOC)

print(sorted(col1))
print(sorted(col2))

assert sorted(col1) == sorted(col2)

assert sorted(dfr_loc_d1.columns.tolist()) == [ID, LOC]
assert sorted(dfr_loc_d2.columns.tolist()) == [ID, LOC]

channelz = pd.unique(dfr[dfr.dataset_id==2]['channel_code']).tolist()
assert channelz == ['BHZ']

channelz = pd.unique(dfr[dfr.dataset_id==1]['channel_code']).tolist()
assert len(channelz) > 1


channelz = pd.unique(dfr_chile['channel_code']).tolist()
# print(sorted(channelz))
assert sorted(channelz) == ['BHE', 'BHN', 'BHZ', 'BLE', 'BLN', 'BLZ']
assert ~(dfr_chile.outlier).any()

print()
print('**********************************')
print('DATASET ID == 2 IS MAGNITUDEENERGY')
print('**********************************')

print('Unique window_type(s) of globalset: ')
print(pd.unique(dfr.window_type))

print('Unique subclass(s) of globalset: ')
print(pd.unique(dfr.subclass))


dfr_chile['window_type'] = ''
dfr_chile['subclass'] = ''
dfr_chile['dataset_id'] = 3

dfr_loc_d1['dataset_id'] = 1
dfr_loc_d2['dataset_id'] = 2

dfr[LOC] = ''

assert sorted(dfr.columns) == sorted(dfr_chile.columns)
assert sorted(dfr_loc_d1.columns.tolist()) == [ID, 'dataset_id', LOC]
assert sorted(dfr_loc_d2.columns.tolist()) == [ID, 'dataset_id', LOC]

# assert sorted(dfr.columns.tolist()) == sorted(_ for _ in dfr_chile.columns if _ != LOC)



globalset columns:
Index(['Segment.db.id', 'dataset_id', 'psd@0.05sec', 'psd@0.1sec',
       'psd@0.2sec', 'psd@0.5sec', 'psd@1sec', 'psd@2sec', 'psd@3sec',
       'psd@5sec', 'psd@9sec', 'outlier', 'subclass', 'window_type',
       'amplitude_ratio', 'event_id', 'station_id', 'event_time',
       'channel_code', 'magnitude', 'distance_km'],
      dtype='object')

loccha oneminutewindows columns
Index(['Segment.db.id', 'location_code'], dtype='object')

loccha magnitudeenergy columns
Index(['Segment.db.id', 'location_code'], dtype='object')

chile columns
Index(['psd@0.05sec', 'psd@0.1sec', 'psd@0.2sec', 'psd@0.5sec', 'psd@1sec',
       'psd@2sec', 'psd@3sec', 'psd@5sec', 'psd@9sec', 'outlier',
       'amplitude_ratio', 'event_id', 'station_id', 'event_time',
       'channel_code', 'location_code', 'magnitude', 'distance_km',
       'Segment.db.id'],
      dtype='object')
['Segment.db.id', 'amplitude_ratio', 'channel_code', 'distance_km', 'event_id', 'event_time', 'magnitude', 'outlie

In [39]:

for df, did in zip([dfr_loc_d1, dfr_loc_d2], [1, 2]):
    print('Merging dataset %d locations into allset' % did)
    dfr = dfr.merge(df, how='left', on=[ID, 'dataset_id'], validate='m:1')
    # now dfr should have LOC + '_x' and LOC + '_y'
    assert LOC + '_x' in dfr.columns and LOC + '_y' in dfr.columns
    tobeset = dfr[ID].isin(df[ID]) & (dfr.dataset_id == did)
    dfr.loc[tobeset, LOC] = dfr[LOC + '_y']
    dfr.loc[~tobeset, LOC] = dfr[LOC + '_x']
    dfr.drop([LOC+'_x', LOC+'_y'], axis=1, inplace=True)

allset = pd.concat([dfr, dfr_chile], sort=False, axis=0, copy=True)

print('\nlocations in dataset 1')
print(pd.unique(allset.loc[allset.dataset_id==1, LOC]))
print('\nlocations in dataset 2')
print(pd.unique(allset.loc[allset.dataset_id==2, LOC]))
print('\nlocations in dataset 3')
print(pd.unique(allset.loc[allset.dataset_id==3, LOC]))

for df, did in zip([dfr_loc_d1, dfr_loc_d2, dfr_chile], [1, 2, 3]):
    locs1 = pd.unique(allset.loc[allset.dataset_id==did, LOC])
    locs2 = pd.unique(df[LOC])
    try:
        assert sorted(locs1) == sorted(locs2)
    except AssertionError:
        assert sorted(locs1) == sorted(locs2.tolist() + [''])


        

# now save files:
# dataframe.to_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/allset.hdf',
#                 format='table', mode='w', key='allset')




Merging dataset 1 locations into globalset
Merging dataset 2 locations into globalset

locations in dataset 1
['01' '00' '']

locations in dataset 2
['' '00' 'BT' '10' '01' '02' '15' '51' '60' '06' '04' '03' '07' '05' '50'
 '55' '20' '30' '40' '80' '70']

locations in dataset 3
['']


In [46]:
print('\ndataset 1 (oneminutewindows) item(s) of first db id')
id_ = dfr_d1[ID].iloc[0]
shared_columns = list(set(dfr_d1.columns) & set(allset.columns))
print(dfr_d1[dfr_d1[ID] == id_][shared_columns].to_string())
print('allset same item(s)')
print(allset[(allset[ID] == id_) & (allset.dataset_id==1)][shared_columns].to_string())

print('\ndataset 2 (magnitudeenergy) item(s) of first db id')
id_ = dfr_d2[ID].iloc[0]
shared_columns = list(set(dfr_d2.columns) & set(allset.columns))
print(dfr_d2[dfr_d2[ID] == id_][shared_columns].to_string())
print('allset same item(s)')
print(allset[(allset[ID] == id_) & (allset.dataset_id==2)][shared_columns].to_string())

print('\ndataset 3 (chile) item(s) of first db id')
id_ = dfr_chile[ID].iloc[0]
shared_columns = list(set(dfr_chile.columns) & set(allset.columns))
print(dfr_chile[dfr_chile[ID] == id_][shared_columns].to_string())
print('allset same item(s)')
print(allset[(allset[ID] == id_) & (allset.dataset_id==3)][shared_columns].to_string())



dataset 1 (oneminutewindows) item(s) of first db id
            event_time    psd@3sec  magnitude    psd@1sec window_type  event_id  psd@0.1sec channel_code  amplitude_ratio  station_id  outlier  Segment.db.id    psd@5sec  distance_km    psd@2sec    psd@9sec  psd@0.5sec  psd@0.2sec  psd@0.05sec
0  2011-01-02 14:40:11 -150.907489        4.2 -135.168397           n     73468 -133.747422          EHN         0.004554         512    False       22740574 -162.001655   106.546261 -145.581222 -168.986145 -129.668991 -134.670527  -135.821416
2  2011-01-02 14:40:11 -130.107018        4.2  -98.268779           s     73468  -96.245629          EHN         0.004554         512    False       22740574 -134.318056   106.546261 -118.620953 -134.014328  -81.439253  -84.732465  -111.035331
3  2011-01-02 14:40:11 -204.174355        4.2 -188.449218           n     73468 -179.188137          EHN         0.004554         512     True       22740574 -208.025795   106.546261 -198.699748 -216.537140 -183.797

In [57]:
from sod.core.dataset import globalset

def selector(d):
    return (d['dataset_id'] == 3) | (d['dataset_id'] == 2) \
        | ((d['dataset_id'] == 1) & (d['subclass'].str.match('^$')) & (~d['outlier'])) \
        | ((d['dataset_id'] == 1) & (d['subclass'].str.contains('INVFILE:')))

dftrain = allset[selector(allset)]
dftest = allset[~selector(allset)]

colz = ['subclass', 'window_type', 'dataset_id', 'outlier', LOC]
print('\nUnique subclasses/window_type/datase_id/outlier/location_code in train:')
for colzz, _ in dftrain.groupby(colz):
    print(str(colzz))

print('\nUnique subclasses/window_type/datase_id/outlier/location_code in test:')
for colzz, _ in dftest.groupby(colz):
    print(str(colzz))




Unique subclasses/window_type/datase_id/outlier/location_code in train:
('', '', 2, False, '')
('', '', 2, False, '00')
('', '', 2, False, '10')
('', '', 2, False, '20')
('', '', 2, False, '50')
('', '', 2, False, '60')
('', '', 2, True, '')
('', '', 2, True, '00')
('', '', 3, False, '')
('', 'n', 1, False, '')
('', 'n', 1, False, '00')
('', 'n', 1, False, '01')
('', 's', 1, False, '')
('', 's', 1, False, '00')
('', 's', 1, False, '01')
('INVFILE:CH.GRIMS.2011-11-09T00:00:00.xml', 'n', 1, True, '')
('INVFILE:CH.GRIMS.2011-11-09T00:00:00.xml', 's', 1, True, '')
('INVFILE:CH.GRIMS.2015-10-30T10:50:00.xml', 'n', 1, True, '')
('INVFILE:CH.GRIMS.2015-10-30T10:50:00.xml', 's', 1, True, '')
('INVFILE:FR.PYLO.2010-01-17T10:00:00.xml', 'n', 1, True, '00')
('INVFILE:FR.PYLO.2010-01-17T10:00:00.xml', 's', 1, True, '00')
('INVFILE:SK.MODS.2004-03-17T00:00:00.xml', 'n', 1, True, '')
('INVFILE:SK.MODS.2004-03-17T00:00:00.xml', 's', 1, True, '')
('INVFILE:SK.ZST.2004-03-17T00:00:00.xml', 'n', 1, Tru

In [58]:
# now save files:
dftrain.to_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/allset_train.hdf',
               format='table', mode='w', key='allset_train')

dftest.to_hdf('/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/allset_test.hdf',
               format='table', mode='w', key='allset_test')
